In [1]:
import gmsh 
import numpy as np

In [2]:
t_n = 3.0
t_l = 0.5
n_l = 3

Lx = 30 # mm
Ly = 10 # mm

E_n = 1.0 # MPa
E_l = [ 1.0, 2.0, 3.0] # MPa


t_t = 2*t_l*n_l + t_n

# Create a box layers 
gmsh.initialize()

gmsh.model.add("compisite_Layers")
# create a fullbox 

addBox = lambda Lz: gmsh.model.occ.addBox( 0  ,  0  , -Lz/2, 
                                              Lx ,  Ly , Lz)
total_box  = addBox(t_t)
gmsh.model.occ.synchronize()


# add planes 
zspan = [2*t_l*i + t_n for i in range(0,n_l,1)] 
zspan = np.array(zspan)/2
zspan = np.concatenate((-zspan, zspan))

wires = []
for z in zspan:
    wireTag = gmsh.model.occ.addRectangle(0, 0, z, Lx, Ly)
    wires.append(wireTag)
gmsh.model.occ.synchronize()

# fragment the box
gmsh.model.occ.fragment([(3, total_box)], [(2, wire) for wire in wires])

gmsh.model.occ.synchronize()


In [3]:
volumen = gmsh.model.getEntities(3)
# sort by z 
center_z = np.array([gmsh.model.occ.getCenterOfMass(3, vol[1])[2] for vol in volumen])
idx = np.argsort(center_z)
volumen = np.array(volumen)[idx]

names = ["Layer_{}".format(abs(i))
         for i in range(-n_l,n_l+1)]
sign = [ "plus" if i > 0 else "minus" for i in range(-n_l,n_l+1)]

names = [name + "_" + s for name,s in zip(names,sign)]
names
# the element that cotain layer 0 replace by nucleo
names[n_l] = "nucleo"

# add physical group
for name, vol in zip(names, volumen):
    gmsh.model.addPhysicalGroup(3, [vol[1]],name= name)

# select all surface which center of mass is in the plane x = 0
# ============================================================
surfaces = gmsh.model.getEntities(2)
surfaces = np.array(surfaces)
center_x = np.array([gmsh.model.occ.getCenterOfMass(2, surf[1])[0] for surf in surfaces])

idx = np.where(center_x == 0)[0]

# add physical group
gmsh.model.addPhysicalGroup(2, [surf[1] for surf in surfaces[idx]], name = "symmetry")

# select all egdes which center of mass is in x = Lx and z=0

edges = gmsh.model.getEntities(1)
edges = np.array(edges)
center_x = np.array([gmsh.model.occ.getCenterOfMass(1, edge[1])[0] for edge in edges])
center_z = np.array([gmsh.model.occ.getCenterOfMass(1, edge[1])[2] for edge in edges])

idx = np.where((center_x == Lx) & (center_z == 0))[0]

# add physical group

gmsh.model.occ.synchronize()

In [4]:
# set size 0.1 of Characteristic Length min
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.01)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 2*t_l)
# mesh 
# near to the symmetry plane, the mesh is finer
field = gmsh.model.mesh.field.add("Distance", 1)
# x = 0 and z = 0

gmsh.model.mesh.field.setAsBackgroundMesh(1)

# set the point of the symmetry plane

#
gmsh.model.mesh.generate(3)
# save inp 
# set order 
gmsh.model.mesh.setOrder(2)
gmsh.write("composite_layers.inp")


In [5]:
from djccx.inp.inp import inp
from Composite.inp.CreateNsetFromElset import CreateNsetFromElset

In [6]:
inp_f = inp("composite_layers.inp")

elset_symmetry = inp_f.select("SYMMETRY","elset")
nset_symmetry = CreateNsetFromElset(inp_f, elset_symmetry, "nset_symmetry")

# select all nodes that have x=Lx and z=0
df_nodes = inp_f.nodes.df
eps = 1e-3

sel_egdes = lambda x,z: df_nodes[(df_nodes["x"] > x - eps) &\
                                 (df_nodes["x"] < x + eps) &\
                                 (df_nodes["z"] > z - eps) &\
                                 (df_nodes["z"] < z + eps)].index


nid = sel_egdes(Lx,-t_t/2)
nset_fixed = inp_f.CreateNsetFromIds(nid, "nset_fixed")

nid = sel_egdes(0,t_t/2)
nset_load = inp_f.CreateNsetFromIds(nid, "nset_load")


#
#
# remove 1d 2d elements
inp_f.remove_by_type(1)
inp_f.remove_by_type(2)

elset_all = inp_f.CreateElsetAll()

# 

materials = []
for i,iEs in enumerate(E_l):
    name_mat = "mat_{}".format(i+1)
    materials.append(inp_f.CreateElasticMaterial(name_mat, iEs, 0.3))

mat_nucleo   = inp_f.CreateElasticMaterial("mat_nucleo", E_n, 0.3)
elset_nucleo = inp_f.select("NUCLEO","elset")
inp_f.CreateSolidSection(elset_nucleo,mat_nucleo)

layer_sel = lambda i: inp_f.select_regex("LAYER_{}.*".format(i),"elset")
for i, iEs in enumerate(E_l):
    for ielset in layer_sel(i+1):
        inp_f.CreateSolidSection(ielset,materials[i])

istep = inp_f.CreateStaticStep()
# 

istep.CreateBoundary(nset_symmetry,1,0.0)
istep.CreateBoundary(nset_fixed,3,0.0)
istep.CreateBoundary(nset_load,3,-1.0)

import os
# create output if not exist
if os.path.exists("output") == False:
    os.mkdir("output")
inp_f.run("output")

PermissionError: [WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'output\\TFP3\\sim\\err.txt'

In [ ]:
inp_f.cards

array([Card (None) :*HEADING, Card (*NODE) :*NODE,
       Card (*ELEMENT) :VOLUME1, Card (*ELEMENT) :VOLUME2,
       Card (*ELEMENT) :VOLUME3, Card (*ELEMENT) :VOLUME4,
       Card (*ELEMENT) :VOLUME5, Card (*ELEMENT) :VOLUME6,
       Card (*ELEMENT) :VOLUME7, Card (*ELSET) :SYMMETRY,
       Card (*ELSET) :LAYER_3_MINUS, Card (*ELSET) :LAYER_2_MINUS,
       Card (*ELSET) :LAYER_1_MINUS, Card (*ELSET) :NUCLEO,
       Card (*ELSET) :LAYER_1_PLUS, Card (*ELSET) :LAYER_2_PLUS,
       Card (*ELSET) :LAYER_3_PLUS, Card (*NSET) :NSET_SYMMETRY,
       Card (*NSET) :NSET_FIXED, Card (*NSET) :NSET_LOAD,
       Card (*ELSETOFELSET) :ALL, Card (*MATERIAL) :MAT_1,
       Card (*MATERIAL) :MAT_2, Card (*MATERIAL) :MAT_3,
       Card (*MATERIAL) :MAT_NUCLEO, Card (*SOLIDSECTION) :SOLID_SECTION,
       Card (*SOLIDSECTION) :SOLID_SECTION,
       Card (*SOLIDSECTION) :SOLID_SECTION,
       Card (*SOLIDSECTION) :SOLID_SECTION,
       Card (*SOLIDSECTION) :SOLID_SECTION,
       Card (*SOLIDSECTION) :SOLI

In [ ]:
layer_sel(3)

[Card (*ELSET) :LAYER_3_MINUS, Card (*ELSET) :LAYER_3_PLUS]

In [ ]:
t_n = 3.0
t_l = 0.5
n_l = 3

Lx = 30 # mm
Ly = 10 # mm

E_n = 1.0 # MPa
E_l = [ 1.0, 2.0, 3.0] # MPa